In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, maximum, average, multiply, concatenate, Reshape, Conv2D, MaxPooling2D
#from tensorflow.python.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()
tf.debugging.set_log_device_placement(True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [2]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.median()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [4]:
X = np.array(X)
#X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
y_train = y_1
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [5]:
hid_layer = [100, 100, 50]
print(hid_layer[-1])

50


In [6]:
_, des_col = des.shape
_, body_col = body.shape
des_col, body_col

(13, 34)

In [7]:
# product fusion
class early_Concat:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi(self):
        # des modality
        des_input = Input(shape = (self.des_shape)) # input = 17, des_shape = 17
        body_input = Input(shape = (self.body_shape)) # input = 30,body_shape = 30
        
        des_fc = Dense(50)(des_input)
        body_fc = Dense(50)(body_input)
        
        des_reshape_2 = Reshape((1,50))(des_fc)
        body_reshape_2 = Reshape((1,50))(body_fc)
        
        dot_layer = dot([des_reshape_2, body_reshape_2], axes = 1, normalize=False)
        dot_reshape = Reshape((50, 50, 1))(dot_layer)
        
        #conv2d = Conv2D(32, (3, 3), activation='relu')(dot_reshape)
        
        res_a = Conv2D(32, (3, 3), activation='relu', padding="same")(dot_reshape)
        res_a_1 = Conv2D(32, (3, 3), activation='relu', padding="same")(res_a)
        res_a_2 = Conv2D(32, (3, 3), activation='relu', padding="same")(res_a_1)      
        res_ab = res_a + res_a_2
        
        res_b = Conv2D(32, (3, 3), activation='relu', padding="same")(res_ab)
        res_b_1 = Conv2D(32, (3, 3), activation='relu', padding="same")(res_b)
        res_b_2 = Conv2D(32, (3, 3), activation='relu', padding="same")(res_b_1)
        res_bc = res_b + res_b_2
        
        res_c = Conv2D(32, (3, 3), activation='relu', padding="same")(res_bc)
        res_c_1 = Conv2D(32, (3, 3), activation='relu', padding="same")(res_c)
        res_c_2 = Conv2D(32, (3, 3), activation='relu', padding="same")(res_c_1)
        res_cc = res_c + res_c_2
        
        
        #conv2d_2 = Conv2D(16, (3, 3), activation='relu')(max_pool_1)
        #max_pool_2 = MaxPooling2D((2, 2))(conv2d_2)
        tensor_flat = Flatten()(res_cc)
        #x = Dense(self.des_shape + self.body_shape , activation = 'relu')(concat)
        #x = BatchNormalization()(x)
        
        x = Dense(hid_layer[2] , activation = 'relu')(tensor_flat)
        x = BatchNormalization()(x)

        #x = Dense(hid_layer[1] , activation = 'relu')(x)
        #x = BatchNormalization()(x)

        x = Dense(hid_layer[2] , activation = 'relu')(x)
        x = BatchNormalization()(x)
        
        x = Dense(self.output_layer , activation = 'sigmoid')(x)
        
        early_fusion = keras.models.Model(inputs = [des_input, body_input], outputs = [x])  #bias_input, 

        return early_fusion

In [8]:
model = early_Concat(des_col, body_col, hid_layer, 1)
earlyfusion= model.Multi()
print(earlyfusion)

In [9]:
earlyfusion.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 50)           700         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           1750        input_2[0][0]                    
______________________________________________________________________________________________

In [10]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score, train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 5, shuffle = True, random_state = 111)
kfold.get_n_splits(X)
print(sm)

SMOTE(random_state=202004)


In [12]:
f_model = early_Concat(des_col, body_col, hid_layer, 1) # input type = integer
f_network = f_model.Multi()

f_network.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
f_history = f_network.fit(x = [des_train, body_train], y = y_train, epochs=50)

y_pred = f_network.predict([des_test[:], body_test[:]])

Epoch 1/50
19/19 [==============================] - 10s 529ms/step - loss: 0.6888 - accuracy: 0.5907
Epoch 2/50
19/19 [==============================] - 10s 533ms/step - loss: 0.6052 - accuracy: 0.7668
Epoch 3/50
19/19 [==============================] - 10s 528ms/step - loss: 0.5321 - accuracy: 0.8532
Epoch 4/50
19/19 [==============================] - 10s 512ms/step - loss: 0.4726 - accuracy: 0.9067
Epoch 5/50
19/19 [==============================] - 9s 491ms/step - loss: 0.4213 - accuracy: 0.9102
Epoch 6/50
19/19 [==============================] - 10s 504ms/step - loss: 0.3925 - accuracy: 0.8981
Epoch 7/50
19/19 [==============================] - 10s 528ms/step - loss: 0.3459 - accuracy: 0.9292
Epoch 8/50
19/19 [==============================] - 10s 514ms/step - loss: 0.3112 - accuracy: 0.9206
Epoch 9/50
19/19 [==============================] - 10s 520ms/step - loss: 0.2840 - accuracy: 0.9413
Epoch 10/50
19/19 [==============================] - 5s 266ms/step - loss: 0.2394 - accuracy

In [16]:
f_y_result = Evaluation(y_pred, y_test)
print(f_y_result.matrix())

공통 
AUC : 0.5338427947598252
AUPRC : 0.9329732629945212
Optimized precision : -0.02290345221889929

function 사용
Accuracy : 0.8875502008032129
Precision(pb) : 0.932755993788632
Recall(pb) : 0.502707423580786
F1 score(pb) : 0.6533128699668507
MCC : 0.01481031823768027
G-mean : 0.21916873521973113

Confusion_matrix 사용 
 [[  1  19]
 [  9 220]]
Accuracy : 0.8875502008032129
Precision : 0.9205020920502092
Recall : 0.9606986899563319
F1 score : 0.9401709401709402
MCC : 0.06347279244720117
G-mean : 0.2191687352197311
TPR : 0.9606986899563319
TNR : 0.05
(0.5338427947598252, 0.9329732629945212, -0.02290345221889929, 0.8875502008032129, 0.932755993788632, 0.502707423580786, 0.6533128699668507, 0.01481031823768027, 0.21916873521973113, 0.8875502008032129, 0.9205020920502092, 0.9606986899563319, 0.9401709401709402, 0.06347279244720117, 0.2191687352197311)


In [11]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [12]:
dict_y = dict()
epochs = 50
class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            
            scaler = StandardScaler()
            scaler1 = StandardScaler()
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            des_train = scaler.fit_transform(des_train)
            body_train = scaler1.fit_transform(body_train)
            
            des_test = scaler.fit_transform(des_test)
            body_test = scaler1.fit_transform(body_test)
            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            print(des_train.shape, body_train.shape)
            
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=epochs)
            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_resample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=epochs)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)
            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_resample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
        
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)
            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_smcw_y

In [13]:
model = early_Concat(des_col, body_col, hid_layer, 1)
earlyfusion= model.Multi()
print(earlyfusion)

early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []
output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)

# sum model training
early_y = output_early.train_based()
#early_sm_y = output_early.train_smote()
#early_cw_y = output_early.train_weight()
#early_smcw_y = output_early.train_sm_weight()

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
(662, 13) (662, 34)
Epoch 1/50
21/21 [==============================] - 3s 25ms/step - loss: 0.7383 - accuracy: 0.5635 0s - loss: 0.7554 - accuracy
Epoch 2/50
21/21 [==============================] - 0s 21ms/step - loss: 0.5992 - accuracy: 0.7253
Epoch 3/50
21/21 [==============================] - 0s 21ms/step - loss:

Epoch 14/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0053 - accuracy: 0.9996
Epoch 15/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0086 - accuracy: 0.9989
Epoch 16/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0064 - accuracy: 0.9991 0s - loss: 0.0051 - accuracy: 
Epoch 17/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0116 - accuracy: 0.9955
Epoch 18/50
21/21 [==============================] - 0s 20ms/step - loss: 0.0063 - accuracy: 0.9980 0s - loss: 0.0064 - accuracy: 0.
Epoch 19/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0073 - accuracy: 0.9993
Epoch 20/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0036 - accuracy: 0.9988 0s - loss: 0.0034 - accuracy: 0.99
Epoch 21/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0225 - accuracy: 0.9958
Epoch 22/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0238 - accuracy: 0.9957
E

21/21 [==============================] - 0s 21ms/step - loss: 0.0187 - accuracy: 0.9963
Epoch 34/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0278 - accuracy: 0.9908
Epoch 35/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0211 - accuracy: 0.9942
Epoch 36/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0171 - accuracy: 0.9974
Epoch 37/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0080 - accuracy: 0.9982 0s - loss: 0.0063 - accuracy: 0.99
Epoch 38/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0084 - accuracy: 0.9961
Epoch 39/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0073 - accuracy: 0.9993
Epoch 40/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0120 - accuracy: 0.9967 0s - loss: 0.0032 - accura
Epoch 41/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0211 - accuracy: 0.9911
Epoch 42/50
21/21 [==============================]

21/21 [==============================] - 1s 21ms/step - loss: 0.0020 - accuracy: 0.9993
Epoch 2/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0031 - accuracy: 1.0000 0s - loss: 0.0031 - accuracy: 1.
Epoch 3/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0155 - accuracy: 0.9938
Epoch 4/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0016 - accuracy: 0.9996ETA: 0s - loss: 8.6132e-04 - accura
Epoch 5/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0013 - accuracy: 0.9999
Epoch 6/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0038 - accuracy: 0.9995ETA: 0s - loss: 5.5448e-04 - accura
Epoch 7/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0025 - accuracy: 0.9990
Epoch 8/50
21/21 [==============================] - 0s 21ms/step - loss: 4.9656e-04 - accuracy: 1.0000
Epoch 9/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0026 - accuracy: 0.9996
Epoch 10/50
2

In [15]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy)
        print("Precision(pb) :", mean_precision) #pb
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb
        print("MCC :", mcc)
        print("G-mean :", g_mean)
        
        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 
        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        print("TPR :", tpr)
        print("TNR :", tnr)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [16]:
early_y_result = Evaluation(predict_change(early_y), y_1)
#early_sm_y_result = Evaluation(early_sm_y, y)
#early_cw_y_result = Evaluation(predict_change(early_cw_y), y)
#early_smcw_y_result = Evaluation(early_smcw_y, y)

print("#early model 결과 \n")
print("\n-----based model-----\n")
print(early_y_result.matrix())

print("\n-----smote model-----\n")
#print(early_sm_y_result.matrix())

print("\n-----class weight model-----\n")
#print(early_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(early_smcw_y_result.matrix())

#early model 결과 


-----based model-----

공통 
AUC : 0.9642398645505795
AUPRC : 0.996171777816365
Optimized precision : 0.390163915478273

function 사용
Accuracy : 0.961352657004831
Precision(pb) : 0.9933097330720786
Recall(pb) : 0.5846745064710719
F1 score(pb) : 0.7360819752229832
MCC : 0.7344969357114279
G-mean : 0.8173240973520715

Confusion_matrix 사용 
 [[ 48  23]
 [  9 748]]
Accuracy : 0.961352657004831
Precision : 0.9701686121919585
Recall : 0.988110964332893
F1 score : 0.9790575916230366
MCC : 0.738468079185454
G-mean : 0.8173240973520715
TPR : 0.988110964332893
TNR : 0.676056338028169
(0.9642398645505795, 0.996171777816365, 0.390163915478273, 0.961352657004831, 0.9933097330720786, 0.5846745064710719, 0.7360819752229832, 0.7344969357114279, 0.8173240973520715, 0.961352657004831, 0.9701686121919585, 0.988110964332893, 0.9790575916230366, 0.738468079185454, 0.8173240973520715)

-----smote model-----


-----class weight model-----


-----smote + class weight model-----



C:\Users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [20]:
early_y_set = []

for i in range(20):
    model = early_Concat(des_col, body_col, hid_layer, 1)
    earlyfusion= model.Multi()
    print(earlyfusion)

    early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []
    output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)

    # concatenate model training
    early_y = output_early.train_based()
    #early_sm_y = output_early.train_smote()
    #early_cw_y = output_early.train_weight()
    #early_smcw_y = output_early.train_sm_weight()
    
    early_y_result = Evaluation(predict_change(early_y), y_1)
    #early_sm_y_result = Evaluation(early_sm_y, y)
    #early_cw_y_result = Evaluation(predict_change(early_cw_y), y)
    #early_smcw_y_result = Evaluation(predict_change(early_smcw_y), y)
    
    early_y_set = np.append(early_y_set, early_y_result.matrix())
    print(len(early_y_set))
    
early_y_np = np.array(early_y_set)

early_y_np = early_y_np.reshape(-1,15)
early_y_np.shape

'\nearly_y_set = []\n\nfor i in range(20):\n    model = early_Concat(des_col, body_col, hid_layer, 1)\n    earlyfusion= model.Multi()\n    print(earlyfusion)\n\n    early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []\n    output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)\n\n    # concatenate model training\n    early_y = output_early.train_based()\n    #early_sm_y = output_early.train_smote()\n    #early_cw_y = output_early.train_weight()\n    #early_smcw_y = output_early.train_sm_weight()\n    \n    early_y_result = Evaluation(predict_change(early_y), y_1)\n    #early_sm_y_result = Evaluation(early_sm_y, y)\n    #early_cw_y_result = Evaluation(predict_change(early_cw_y), y)\n    #early_smcw_y_result = Evaluation(predict_change(early_smcw_y), y)\n    \n    early_y_set = np.append(early_y_set, early_y_result.matrix())\n    print(len(early_y_set))\n    \nearly_y_np = np.array(early_y_set)\n\nearly_y_np = early_y_np.reshape(-1,15)\nearly_y_np.shape\n

In [17]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [19]:
result(early_y_np)